В задании вам понадобится собрать генеративную модлель для языка

---

# LSTM (7 баллов)

В данной части нужно реализовать модель с ипользованием LSTM

In [29]:
!pip install --quiet sentencepiece datasets transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [30]:
import random
import torch
import numpy as np
from tqdm.notebook import tqdm, trange
from sklearn.model_selection import train_test_split
import os

import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

In [31]:
from transformers import BertTokenizerFast


# Добавьте код для подготовки данных

def read_file(path):
    with open(path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text


def remove_consecutive_spaces(string):
    return ' '.join(string.split())


def chars(f, t):
    return list(map(chr, range(ord(f), ord(t) + 1)))


valid_chars = set(chars('а', 'я') + ['ё'] + chars('a', 'z') + chars('0', '9')
                  + list("<>(){}" + ".,\"!?;:-*—\'"))


def clean_text(text):
    text = text.lower()
    text = text.replace('\n', ' ')
    text = ''.join([char if char in valid_chars else ' ' for char in text])
    text = remove_consecutive_spaces(text)
    return text


def load_texts(paths):
    texts = [clean_text(read_file(path)) for path in paths]

    return texts


def get_all_files(root):
    files_paths = []
    for root, dir, files in os.walk(root):
        for file in files:
            if file.endswith(".txt"):
                files_paths.append(os.path.join(root, file))
    return files_paths


tokenizer = BertTokenizerFast('vocab.txt')


def encode(text):
    return tokenizer.encode(text, add_special_tokens=False)


def combine_texts(texts):
    tokens = [token for text in texts for token in encode(text)]

    return tokens


class TextDataset(torch.utils.data.Dataset):
    def __init__(
            self,
            texts,
            offset,
            max_seq_len,
    ):
        self.texts = texts
        self.offset = offset
        self.max_seq_len = max_seq_len
        self.tokens = combine_texts(texts)
        self.slices = []
        for i in range(0, len(self.tokens) - max_seq_len, offset):
            self.slices.append(self.tokens[i:i + max_seq_len + 1])

    def __getitem__(self, idx):
        return self.slices[idx]

    def __len__(self):
        return len(self.slices)

In [32]:
device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
device

device(type='mps')

Ниже вам нужно реализовать модель, которая по началу последовательности предсказывает следующий токен.
*   Модель получает на вход последовательность токенов, прогоняет её через LSTM и выдает вероятности следующего токена.  
*   Используйте LSTM из pytorch
*   Не забудьте про `batch_first`


In [33]:
class RnnGenerator(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_layers):
        super().__init__()
        self.embed = torch.nn.Embedding(vocab_size, embed_dim)
        self.rnn = torch.nn.LSTM(embed_dim, embed_dim, num_layers, batch_first=True)
        self.fc = torch.nn.Linear(embed_dim, vocab_size)

    #x: (batch_size, seq_len)
    def forward(self, x):
        x = self.embed(x)
        x, (h_n, c_n) = self.rnn(x)
        x = self.fc(x)

        return x

In [34]:
tokenizer.vocab_size

8000

In [35]:
vocab_size = tokenizer.vocab_size

model = RnnGenerator(vocab_size, 512, 2)

In [36]:
pad_token_idx = 0
sep_token_idx = 3
max_seq_len = 256
batch_size = 16


def collate_batch(batch):
    src_list = []
    for src in batch:
        src_list.append(torch.tensor(src))

    return pad_sequence(src_list, padding_value=pad_token_idx, batch_first=True)

In [37]:
paths = get_all_files("books_txt")
paths.sort()
print(f'files cnt: {len(paths)}')

files cnt: 749


In [38]:
# Выделим небольшой процент данных для теста
paths_train, paths_test = train_test_split(paths, test_size=0.03, random_state=12345)

texts_train = load_texts(paths_train)
texts_test = load_texts(paths_test)

In [39]:
train_data = TextDataset(
    texts_train,
    max_seq_len // 2,
    max_seq_len=max_seq_len,
)
train_dataloader = DataLoader(train_data, batch_size, shuffle=True, collate_fn=collate_batch)

In [40]:
test_data = TextDataset(
    texts_test,
    max_seq_len * 2,
    max_seq_len=max_seq_len,
)  # Что бы не ждать будем использовать большой offset
test_dataloader = DataLoader(test_data, batch_size, collate_fn=collate_batch)

In [41]:
print(len(train_data))

273864


In [42]:
batch = next(iter(train_dataloader))
model(batch[:, :-1]).shape

torch.Size([16, 256, 8000])

In [43]:
_ = model.to(device)

In [44]:
learning_rate = 0.0005

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [45]:
criterion = torch.nn.CrossEntropyLoss(ignore_index=pad_token_idx)

Реализуйте обучение модели:
*  Не забудьте сдвинуть src и trg относительно друг друга.
*  Не забудьте про `clip_grad_norm_`
*  Данных очень много, для отладки лучше проходить только часть данных иначе этоха будет очень длинной

In [46]:
# Реализуйте обучение, 
# можете не отстанавливать цикл раньше так как выборка большая.
def train_epoch(model, callback):
    model.train()
    losses = []
    n_inter = 0

    for batch in tqdm(train_dataloader):
        src = batch[:, :-1].to(device)
        trg = batch[:, 1:].to(device)

        optimizer.zero_grad()
        output = model(src)
        loss = criterion(output.transpose(1, 2), trg)
        losses.append(loss.item())

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        n_inter += 1
        if n_inter % 500 == 0:
            callback(np.mean(losses))
            losses = []



In [47]:
def eval_model(model):
    model.eval()
    losses = []

    for batch in test_dataloader:
        src = batch[:, :-1].to(device)
        trg = batch[:, 1:].to(device)

        with torch.no_grad():
            logits = model(src)
            loss = criterion(logits.transpose(1, 2), trg)
        losses.append(loss.item())

    return np.mean(losses)

In [48]:
eval_model(model)

np.float64(8.992234655788966)

Получите `loss < 5.0` на тестовой выборке.

Если модель обучается слишком быстро до значений <1.0 вы что-то напутали с данными.

In [49]:
def callback(train_loss):
    eval_loss = eval_model(model)
    model.train()
    print(f'Epoch: {epoch+1:02} | train_loss = {train_loss:.5f}, eval_loss = {eval_loss:.5f}')

for epoch in trange(1):
    train_loss = train_epoch(model, callback)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/17117 [00:00<?, ?it/s]

Epoch: 01 | train_loss = 6.77620, eval_loss = 6.18671
Epoch: 01 | train_loss = 6.03131, eval_loss = 5.81084
Epoch: 01 | train_loss = 5.72296, eval_loss = 5.56409
Epoch: 01 | train_loss = 5.49744, eval_loss = 5.35311
Epoch: 01 | train_loss = 5.32138, eval_loss = 5.18744
Epoch: 01 | train_loss = 5.16289, eval_loss = 5.05937
Epoch: 01 | train_loss = 5.04530, eval_loss = 4.96199
Epoch: 01 | train_loss = 4.94420, eval_loss = 4.87713
Epoch: 01 | train_loss = 4.85471, eval_loss = 4.80751
Epoch: 01 | train_loss = 4.79700, eval_loss = 4.74784
Epoch: 01 | train_loss = 4.72877, eval_loss = 4.69445
Epoch: 01 | train_loss = 4.67384, eval_loss = 4.65188
Epoch: 01 | train_loss = 4.63009, eval_loss = 4.61307
Epoch: 01 | train_loss = 4.59660, eval_loss = 4.58135
Epoch: 01 | train_loss = 4.54663, eval_loss = 4.55238
Epoch: 01 | train_loss = 4.52166, eval_loss = 4.52584
Epoch: 01 | train_loss = 4.49308, eval_loss = 4.49595
Epoch: 01 | train_loss = 4.46308, eval_loss = 4.47770
Epoch: 01 | train_loss = 4.4

In [51]:
save_path = '/Users/ddulaev/PycharmProjects/dltheory/lab3/lstm_model.pth'
torch.save(model.state_dict(), save_path)

# Генерация текста (5 баллов)

Реализуйте функцию, которая продолжает текст.
1.   Переведите строчку в токены
2.   Реализуйте код который предсказывает вероятность следующей буквы
3.   Семплируйте следующую букву
4.   Повторяйте пункты 2-3 в цикле
5.   Преобразуйте токены в строчку



In [52]:
from transformers import GPT2TokenizerFast
from tokenizers.decoders import ByteLevel, WordPiece
from transformers import BertTokenizerFast

decoder = WordPiece()

In [53]:
def continues_sentence(sentence, model, max_len = 30):
    model.eval()

    tokens = list(encode(sentence.lower()))

    input_tensor = torch.tensor(tokens).unsqueeze(0).to(device)

    for _ in range(max_len):
        with torch.no_grad():
            output = model(input_tensor)[:, -1, :]
            softmax = torch.softmax(output, dim=-1)

            token = torch.multinomial(softmax, num_samples=1).item()
            tokens.append(token)

            input_tensor = torch.tensor(tokens).unsqueeze(0).to(device)

    return decoder.decode(tokenizer.convert_ids_to_tokens(tokens))

In [54]:
continues_sentence("Я помню чудное мгновенье", model)

'я помню чудное мгновенье - золотое ; сверху, другая - сила, светлость речи он ум гнева, в сражении рютона, кружась'

In [55]:
continues_sentence("Мой дядя самых честных правил,", model)

'мои дядя самых честных правил, а есть тоже сперва глупыми или бурю ; окохе их у нас глаза от устали мы вас видима, да и всего больше в нов'

In [56]:
continues_sentence("Четыре года потратил Деонардо на", model)

'четыре года потратил деонардо на англия. биует с большим гером он был совершенно холоден, и раблики с москвами решетками, выставить'

In [57]:
continues_sentence("Если сила плохих людей в том, что они вместе, то хорошим людям, чтобы стать силой, надо", model)

'если сила плохих людеи в том, что они вместе, то хорошим людям, чтобы стать силои, надо видеть ему, настолько собственно благодарить того, что настоящее актеры и печать, когда они не укорлись и пожалело, но так и'